In [8]:
import numpy as np

# Grid Properties

In [2]:
N, L = 4, 1.

In [3]:
dx = L/N

In [4]:
dx

0.25

In [5]:
area = 0.02

# Reservoir Properties

In [6]:
perm = 50
poro = 0.2

# Fluid Property

In [7]:
visc = 0.01

def comp(pressure):
    return 1./pressure

In [9]:
PL = 200
PR = 14.7
Pinit = np.full(N,14.7)

In [10]:
Pinit

array([14.7, 14.7, 14.7, 14.7])

# Numerical Settings

In [11]:
tstep = 1e-5

In [12]:
nstep = 3

# Block Calculations

In [77]:
conv1 = 1.06235016e-14*0.3048/0.001
conv3 = 1.06235016e-14/1.4503774389728e-7*24*60*60

In [78]:
trans = perm*area/dx/visc*conv3

In [91]:
T = np.zeros((N,N))
J = np.zeros((N,N))
Q = np.zeros((N,1))

for i in range(N):
    if i==0:
        T[i,i] = trans
        T[i,i+1] = -trans
        J[i,i] = 2*trans
        Q[i,0] = 2*trans*PL
    elif i==N-1:
        T[i,i] = trans
        T[i,i-1] = -trans
        J[i,i] = 2*trans
        Q[i,0] = 2*trans*PR
    else:
        T[i,i] = 2*trans
        T[i,i-1] = -trans
        T[i,i+1] = -trans

In [99]:
def storage(pressure):
    A = np.zeros((N,N))
    accum = area*poro*dx*comp(pressure)/tstep
    for i in range(N):
        A[i,i] = accum[i]
    return A

In [100]:
storage(Pinit)

array([[6.80272109, 0.        , 0.        , 0.        ],
       [0.        , 6.80272109, 0.        , 0.        ],
       [0.        , 0.        , 6.80272109, 0.        ],
       [0.        , 0.        , 0.        , 6.80272109]])

In [94]:
J

array([[5.06279545, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 5.06279545]])

In [95]:
A

array([[6.80272109, 0.        , 0.        , 0.        ],
       [0.        , 6.80272109, 0.        , 0.        ],
       [0.        , 0.        , 6.80272109, 0.        ],
       [0.        , 0.        , 0.        , 6.80272109]])

In [118]:
P = Pinit.copy()

for t in range(nstep):
    Pk = P.copy()
    for k in range(100):
        A = storage(Pk)
        F = -np.matmul((T+J+A),Pk)+np.matmul(A,P)+Q.flatten()
        R = np.linalg.norm(F); print(k+1,R)
        if R<1e-6:
            break
        Pk = np.linalg.solve((A+T+J),(np.matmul(A,P)+Q.flatten()))
        # print(Pk)
    P = Pk.copy()
    break

1 938.135997792843
2 382.90288849687494
3 78.45722872411923
4 24.749252903844766
5 8.887181108343485
6 3.247652151919296
7 1.1801584783961532
8 0.42631294878456333
9 0.1535034555127361
10 0.055189741900053865
11 0.019829475796517445
12 0.007122604310859925
13 0.002558067035417075
14 0.0009186737312152341
15 0.00032991367396556314
16 0.0001184772046922548
17 4.254683827470127e-05
18 1.5279139972343016e-05
19 5.486939051706792e-06
20 1.9704309057754455e-06
21 7.076071010690716e-07


In [109]:
A

array([[0.56549092, 0.        , 0.        , 0.        ],
       [0.        , 0.76621013, 0.        , 0.        ],
       [0.        , 0.        , 1.1878248 , 0.        ],
       [0.        , 0.        , 0.        , 2.6411357 ]])